<a href="https://colab.research.google.com/github/Hilbertrui/Hibertrui/blob/main/Copy_of_roboflow_yolov8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

[YOLOv8 of roboflow](https://colab.research.google.com/github/QIN2DIM/hcaptcha-challenger/blob/main/automation/roboflow_yolov8.ipynb)

In [ ]:
#检查GPU状态

In [ ]:
!nvidia-smi

In [ ]:
#安装ONNX相关库和YOLOv8的ultralytics库

In [ ]:
!pip install -U onnx onnxruntime-gpu onnxruntime
!pip install ultralytics==8.0.202

In [ ]:
#训练AI

In [ ]:
import os
import shutil
from pathlib import Path

#初始化yolo模型
from ultralytics import YOLO
HOME = os.getcwd()
%cd {HOME}
challenge_dir_name = "animal-head-2310-1"
output_model_name = "head_of_the_animal_2311_yolov8s.onnx"

#yolo的路径
this_dir = Path(os.path.abspath("__file__")).parent
yolo_data_dir = this_dir.joinpath(challenge_dir_name)
yolo_inspect_path = yolo_data_dir.joinpath("data.yaml")

#训练
model = YOLO("yolov8s.pt")  # detection
# model = YOLO("yolov8s-seg.pt") # segment
model.train(data=str(yolo_inspect_path), epochs=300, imgsz=640)

In [ ]:
#训练好的模型导出为ONNX

In [ ]:
# detection
yolo_best_pt_path = this_dir.joinpath("runs/detect/train/weights/best.pt")
yolo_onnx_path = this_dir.joinpath("runs/detect/train/weights/best.onnx")

# segment
# yolo_best_pt_path = this_dir.joinpath("runs/segment/train/weights/best.pt")
# yolo_onnx_path = this_dir.joinpath("runs/segment/train/weights/best.onnx")

model_path = this_dir.joinpath(output_model_name)

# export to onnx
YOLO(str(yolo_best_pt_path)).export(format="onnx", simplify=True, int8=True)

# copy to current work dir
shutil.copyfile(yolo_onnx_path, model_path)

In [ ]:
#上传到Github

In [ ]:
import locale

locale.getpreferredencoding = lambda d=True: "UTF-8"

!pip install PyGithub

from github import Auth, Github, GithubException

auth = Auth.Token(os.getenv("GITHUB_TOKEN", ""))
repo = Github(auth=auth).get_repo("QIN2DIM/hcaptcha-challenger")
modelhub_title = "ONNX ModelHub"


def quick_development(pending_onnx_path: Path):
    if not pending_onnx_path.exists() or not pending_onnx_path.name.endswith(".onnx"):
        return

    for release in repo.get_releases():
        if release.title != modelhub_title:
            continue
        try:
            res = release.upload_asset(path=str(pending_onnx_path))
        except GithubException as err:
            if err.status == 422:
                print(
                    "The model file already exists, please manually replace the file with the same"
                    f" name - url={repo.releases_url}"
                )
        except Exception as err:
            print(err)
        else:
            print(
                f"Model file uploaded successfully - name={res.name} url={res.browser_download_url}"
            )


quick_development(pending_onnx_path=model_path)